In [1]:
#!pip install pyarrow
#!pip install tensorflow

In [1]:
import numpy as np
import gc
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset,TensorDataset
import pandas as pd
import pyarrow
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [27]:
exercise_set_mapping = {exercise_set: index for index, exercise_set in enumerate(df[['Exercise', 'Set']].drop_duplicates().itertuples(index=False))}

NameError: ignored

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/prep_dataset.parquet', engine='pyarrow')


In [21]:
#indices = df.index
#labels = df.columns

X = df.drop(['Participant', 'Exercise', 'Set', 'Camera'], axis=1)
y = df[['Exercise','Set']]

del df

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False, random_state=42)
#del X
#del y_exercise
gc.collect()
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(np.array([exercise_set_mapping[tuple(y)] for y in y_train.values]), dtype=torch.int64)

X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(np.array([exercise_set_mapping[tuple(y)] for y in y_test.values]), dtype=torch.int64)

In [42]:
print(y_train.unique())

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37])


In [43]:
get_mlp = lambda: torch.nn.Sequential(

    torch.nn.Flatten(),
    torch.nn.Linear(100, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 38),
)

get_cnn = lambda: torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Conv1d(1, out_channels=32, kernel_size=3, stride=1, padding=1),
    torch.nn.ReLU(),
    torch.nn.MaxPool1d(kernel_size=2, stride=2),
    torch.nn.Linear(32 * 50, 7),
)

In [44]:

"""
Return a modified dataset cls that can insert MNIST like images into larger
frames with an option for random shifts, scrambling the images in a
consistent way (using the same shuffling for all images) and adding random
Gaussian noise (to the base data, noise is always the same for a given
image).
"""

class CustomDataset(Dataset):
    def __init__(self, x_train, y_train, transform=None):
        self.x_train = x_train
        self.y_train = y_train
        self.transform = transform

    def __len__(self):
        return len(self.x_train)

    def __getitem__(self, idx):
        sample = {'input': self.x_train[idx], 'label': self.y_train[idx]}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [45]:
learning_rate = 1e-3
num_epochs = 10

# Create an instance of the custom dataset
custom_dataset = CustomDataset(X_train, y_train)

# Create a DataLoader
batch_size = 64
custom_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

model = get_mlp()
model.train()
#model_path = 'MLP.path'
#model.load_state_dict(torch.load(model_path))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [46]:
#def train_epoch()
for epoch in range(num_epochs):
    losses = []
    for batch in custom_dataloader:
        inputs, labels = batch['input'], batch['label']
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {np.mean(losses):.4f}')

Epoch [1/10], Loss: 1.3017
Epoch [2/10], Loss: 0.9446
Epoch [3/10], Loss: 0.8102
Epoch [4/10], Loss: 0.7354
Epoch [5/10], Loss: 0.6851
Epoch [6/10], Loss: 0.6500
Epoch [7/10], Loss: 0.6222
Epoch [8/10], Loss: 0.5994
Epoch [9/10], Loss: 0.5815
Epoch [10/10], Loss: 0.5679


In [14]:
torch.save(model.state_dict(),'MLP.path')

In [48]:
model.eval()

# Make predictions on the test set
with torch.no_grad():
    predictions = model(X_test)

# Convert predictions to class labels
predicted_labels = torch.argmax(predictions, dim=1)

# Convert tensors to numpy arrays
y_test_np = y_test.numpy()
predicted_labels_np = predicted_labels.numpy()

# Print accuracy
accuracy = accuracy_score(y_test_np, predicted_labels_np)
print(f'Accuracy: {accuracy:.4f}')

# Print classification report
print('Classification Report:')
print(classification_report(y_test_np, predicted_labels_np, labels=range(38)))

Accuracy: 0.3692
Classification Report:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.41      0.29      0.34     26248
           1       0.26      0.42      0.32     13764
           2       0.43      0.37      0.40     19428
           3       0.43      0.53      0.48     25064
           4       0.11      0.15      0.13      8180
           5       0.00      0.00      0.00         0
           6       0.33      0.45      0.38     32228
           7       0.00      0.00      0.00         0
           8       0.03      0.01      0.02      7208
           9       0.48      0.37      0.42     14832
          10       0.45      0.72      0.55     14128
          11       0.41      0.25      0.31     13304
          12       0.43      0.55      0.48     20076
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.46      0.63      0.53     14428
          16       0.36      0.27      0.31      9292
          17       0.56    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
learning_rate = 1e-3
num_epochs = 1


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Create an instance of the custom dataset
#X_train, y_train = X_train.to(device), y_train.to(device)

custom_dataset = TensorDataset(X_train, y_train)

# Create a DataLoader
batch_size = 64
custom_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

model = get_cnn()
model.train()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
for epoch in range(num_epochs):
    losses = []
    for batch in custom_dataloader:
        inputs, labels = batch['input'], batch['label']
        print(inputs.shape)
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {np.mean(losses):.4f}')

TypeError: ignored